In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!pip install tensorflow # Install tensorflow if not already installed
import tensorflow as tf # Import tensorflow as usual
print(tf.__version__) # Print version to confirm successful installation

2.18.0


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data=pd.read_csv(r'/content/drive/MyDrive/train.csv',sep=',',header=0, index_col=False, encoding='latin-1')

In [6]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [7]:
data.isnull().sum()

,0
qid,0
question_text,0
target,0


In [8]:
for col in ['question_text','target']:
  print(col,',',data[col].value_counts())

question_text , question_text
Who wins in a battle between a Wolverine and a Puma?                                                                         1
How did Quebec nationalists see their province as a nation in the 1960s?                                                     1
Do you have an adopted dog, how would you encourage people to adopt and not shop?                                            1
Why does velocity affect time? Does velocity affect space geometry?                                                          1
How did Otto von Guericke used the Magdeburg hemispheres?                                                                    1
                                                                                                                            ..
Can we use our external hard disk as a OS as well as for data storage.will the data be affected?                             1
I am 30, living at home and have no boyfriend. I would love a boyfriend and my ow

In [9]:
data.dtypes

,0
qid,object
question_text,object
target,int64


In [10]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

# Import TensorFlow and then the Keras modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer # Changed import statement
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from tensorflow.keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [11]:
embeding_index={}

f=open('/content/drive/MyDrive/glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [12]:
len(embeding_index['hello'])

300

In [13]:
X=data['question_text']
Y=data['target']

In [14]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [15]:
from sklearn.utils import class_weight

# Calculate class weights
calculated_class_weight = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# Assign the result to a new variable 'calculated_class_weight'
print(calculated_class_weight)

# Create a dictionary from the calculated class weights
class_weight_dict = dict(enumerate(calculated_class_weight))
# Using 'calculated_class_weight' instead of 'class_weight'
print(class_weight_dict)

[0.53294267 8.08894067]
{0: np.float64(0.5329426741976254), 1: np.float64(8.088940670093516)}


In [16]:
x_train.head()

,question_text
944989,Which advertising providers can I use on my da...
1140962,Who created the Laws of the Universe?
541127,Why does lateral stretching of my thoracic spi...
213203,Why isn't Quora trying to stimulate a higher l...
819085,How can I protect myself from falling in love ...


In [17]:
from nltk import word_tokenize

In [18]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [20]:
np.percentile(sent_lens,99)

np.float64(44.0)

In [21]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [22]:
vocab_size

196296

In [23]:
seq_train_matrix.shape

(1044897, 44)

In [24]:
x_train.shape

(1044897,)

In [25]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [26]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [27]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [28]:
embedding_matrix.shape

(196297, 300)

In [29]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32335001,  0.41314   , -0.058101  , ..., -0.50889999,
        -0.59205002,  0.42530999]])

In [58]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,
                weights=[embedding_matrix],trainable=False, mask_zero=True)(inputs) # Pass `mask_zero` as a named argument
lstm_layer1 = LSTM(512, use_cudnn=False)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [59]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 44, 300)   │ 58,889,100 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 44)        │          0 │ text_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 512)       │  1,665,024 │ embedding_2[0][0… │
│                     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 512)       │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256)       │    131,328 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 256)       │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │     32,896 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 25)        │      3,225 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 25)        │          0 │ dense_10[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         26 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 60,721,599 (231.63 MB)

 Trainable params: 1,832,499 (6.99 MB)

 Non-trainable params: 58,889,100 (224.64 MB)

In [32]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [60]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [61]:
import os
checkpoint_path = "training_1/cp.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1)

In [62]:
ml = model.fit(seq_train_matrix,y_train,
              epochs=2,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1500,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [cp_callback])

Epoch 1/2
697/697 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.8501 - loss: 0.3434
Epoch 1: saving model to training_1/cp.keras
697/697 ━━━━━━━━━━━━━━━━━━━━ 210s 294ms/step - accuracy: 0.8501 - loss: 0.3434 - val_accuracy: 0.8731 - val_loss: 0.2948
Epoch 2/2
697/697 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.8876 - loss: 0.2594
Epoch 2: saving model to training_1/cp.keras
697/697 ━━━━━━━━━━━━━━━━━━━━ 194s 279ms/step - accuracy: 0.8876 - loss: 0.2594 - val_accuracy: 0.9043 - val_loss: 0.2359


In [63]:
model.save('model_f.h5')

In [64]:
model.save('my_model.keras')

In [66]:
saved_model = load_model('my_model.keras')
